In [1]:
import requests
import re
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
import time
import os
import json

In [2]:
user_agent = {'User-agent': 'Mozilla/5.0'}

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

driver = webdriver.Chrome(chromedriver)

In [3]:
def get_film_data(film_id):
    
    delimiter = '+_+'
    
    film_url = "http://www.imdb.com/title/" + film_id
    driver.get(film_url)
    
    response = requests.get(film_url, headers = user_agent)
    html = response.text
    soup = bs(html, 'lxml')
    
    film_dict = {}
    
    try:
        genre_selector = "//*[@id='titleStoryLine']//a[contains(@href, '/genre/')]"
        film_dict['genres'] = ''
        for genre in driver.find_elements_by_xpath(genre_selector):
            film_dict['genres'] = film_dict['genres'] + delimiter + genre.text
    except NoSuchElementException:
        film_dict['genres'] = None
        
    try:
        title_selector = "//*[@id='title-overview-widget']//*[@itemprop='name']"
        film_dict['title'] = driver.find_element_by_xpath(title_selector).text
    except NoSuchElementException:
        film_dict['title'] = None
    
    try:
        year_selector = '//*[@id="titleYear"]'
        film_dict['year'] = driver.find_element_by_xpath(year_selector).text.split('(')[1].split(')')[0]
    except NoSuchElementException:
        film_dict['year'] = None  
    
    try:
        rating_selector = "//*[@id='title-overview-widget']//*[@itemprop='ratingValue']"
        film_dict['rating'] = driver.find_element_by_xpath(rating_selector).text
    except NoSuchElementException:
        film_dict['rating'] = None  
   
    try:
        votes_selector = "//*[@id='title-overview-widget']//*[@itemprop='ratingCount']"
        film_dict['votes'] = driver.find_element_by_xpath(votes_selector).text
    except NoSuchElementException:
        film_dict['votes'] = None     
    
    try:
        directors_selector = "//*[@id='title-overview-widget']//*[@itemprop='director']"
        film_dict['directors'] = ''
        for director in driver.find_elements_by_xpath(directors_selector):
            film_dict['directors'] = film_dict['directors'] + delimiter + director.text.split('(')[0].strip()
    except NoSuchElementException:
        film_dict['directors'] = None    
  
    try:
        writers_selector = "//*[@id='title-overview-widget']//*[@itemprop='creator']"
        film_dict['writers'] = ''
        for writer in driver.find_elements_by_xpath(writers_selector):
            film_dict['writers'] = film_dict['writers'] + delimiter + writer.text.split('(')[0].strip()
    except NoSuchElementException:
        film_dict['writers'] = None     

    try:
        actors_selector = "//*[@id='titleCast']//*[@itemprop='name']"
        film_dict['actors'] = ''
        for actor in driver.find_elements_by_xpath(actors_selector):
            film_dict['actors'] = film_dict['actors'] + delimiter + actor.text
    except NoSuchElementException:
        film_dict['actors'] = None          

    try:
        keywords_selector = "//*[@class='see-more inline canwrap']//*[@itemprop='keywords']"
        film_dict['keywords'] = ''
        for keyword in driver.find_elements_by_xpath(keywords_selector):
            film_dict['keywords'] = film_dict['keywords'] + delimiter + keyword.text
    except NoSuchElementException:
        film_dict['keywords'] = None       

    try:
        mpaa_selector = "//meta[@itemprop='contentRating']"
        film_dict['mpaa'] = driver.find_element_by_xpath(mpaa_selector).get_attribute('content')
    except NoSuchElementException:
        film_dict['mpaa'] = None      

    try:
        release_date_selector = "//meta[@itemprop='datePublished']"
        film_dict['release_date'] = driver.find_element_by_xpath(release_date_selector).get_attribute('content')
    except NoSuchElementException:
        film_dict['release_date'] = None     

    try:
        languages_selector = "//*[@id='titleDetails']//a[contains(@href, 'language')]"
        film_dict['languages'] = ''
        for language in driver.find_elements_by_xpath(languages_selector):
            film_dict['languages'] = film_dict['languages'] + delimiter + language.text
    except NoSuchElementException:
        film_dict['languages'] = None   
    
    try:
        countries_selector = "//*[@id='titleDetails']//a[contains(@href, 'country_of_origin')]"
        film_dict['countries'] = ''
        for country in driver.find_elements_by_xpath(countries_selector):
            film_dict['countries'] = film_dict['countries'] + delimiter + country.text
    except NoSuchElementException:
        film_dict['countries'] = None 
    
    try:
        runtime_selector = "//*[@id='titleDetails']//*[@itemprop='duration']"
        film_dict['runtime'] = int(driver.find_element_by_xpath(runtime_selector).text.split(' ')[0])
    except NoSuchElementException:
        film_dict['runtime'] = None 
     
    film_dict['budget'] = None
    film_dict['gross_usa'] = None
    for h4 in soup.find_all('h4'):
        if "Budget:" in h4:
            film_dict['budget'] = h4.next_sibling        
        if "Gross USA:" in h4:
            film_dict['gross_usa'] = h4.next_sibling
            
    return film_dict

In [4]:
with open("film_ids.json","r") as f:
      data = f.read()
        
film_ids = json.loads(data)

In [5]:
dict_list = []
dummy = 0

for film_id in film_ids[30000:31000]:
    print(dummy)
    dict_list.append(get_film_data(film_id))
    dummy = dummy + 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [6]:
import pandas as pd

data = json.dumps(dict_list)

with open("film_data_31.json","w") as f:
      f.write(data)

with open("film_data_31.json","r") as f:
      data = f.read()
        
d = json.loads(data)

df = pd.DataFrame(d)

df

,actors,budget,countries,directors,genres,gross_usa,keywords,languages,mpaa,rating,release_date,runtime,title,votes,writers,year
0,+_+Michael Bole+_+Sunny Osman+_+Ashley Uberti+...,None,+_+USA,+_+Vitaliy Versace,+_+Crime+_+Thriller,None,+_+island+_+mass murder+_+based on real events,+_+English,None,3.6,2012-07-22,60.0,Utoya Island (2012),90,+_+Vitaliy Versace,2012
1,+_+Cara Roberts+_+Chad Phillips+_+Jessica Hain...,None,+_+South Africa+_+Greece+_+USA,+_+Harry Patramanis,+_+Drama,None,,+_+English,None,6.3,2013-01-18,96.0,Fynbos (2012),51,"+_+Jonathan Glatzer,+_+Harry Patramanis",2012
2,+_+Matthew Rhode+_+Collin Ware+_+Ryan Andrew B...,None,+_+USA,+_+Darren Marshall,+_+Comedy+_+Drama,None,,+_+English,NOT RATED,7.1,2012,99.0,The Kings of Yorktown (2012),218,+_+Darren Marshall,2012
3,+_+Anastacia+_+Hali Mason+_+Laural Merlington+...,"$3,000,000\n",+_+USA,+_+Valerio Zanoli,+_+Comedy+_+Drama+_+Family,None,+_+weight+_+courage+_+dream+_+fat+_+diet,+_+English,None,6.4,2013-02,79.0,All You Can Dream (2012),42,+_+Valerio Zanoli,2012
4,+_+Cort Cunningham+_+Tristan John Gluth+_+Cono...,$500\n,+_+USA,+_+Ryland Rhodes,+_+Comedy,None,,+_+English,None,None,2012-07-23,62.0,The Rip Stick Retards Movie (2012),None,"+_+Joseph Ortiz,+_+Ryland Rhodes",2012
5,+_+Kris Lemche+_+Brooke Nevin+_+Joey Kern+_+Da...,None,+_+USA,+_+Jordan Galland,+_+Comedy,None,+_+superhero+_+motel+_+motel room+_+reference ...,+_+English,R,5.2,2012-11-20,80.0,Alter Egos (2012),"1,373",+_+Jordan Galland,2012
6,+_+Steve Bencich+_+Tony Besson+_+Hayley Derryb...,"$150,000\n",+_+USA,+_+Martin Wichmann Andersen,+_+Horror,None,+_+ghost+_+found footage+_+talking to the came...,+_+English,R,4.0,2012-07-24,84.0,100 Ghost Street: The Return of Richard Speck ...,842,,2012
7,+_+Kelsey Tanaka+_+Ryan N. Taylor+_+Eryn Moore...,"$30,000\n",+_+USA,+_+Jay Gerren,+_+Drama,None,+_+christian+_+spirituality,+_+English,UNRATED,4.9,2012-07-24,80.0,The Young Believers (2012),14,+_+Jay Gerren,2012
8,+_+Adam Pitzler+_+Joshua Hudson+_+Ray Tansy+_+...,"$20,000,000\n",+_+USA,+_+Adam Pitzler,+_+Action+_+Comedy,None,,+_+English,None,7.7,2012-07-24,129.0,Manhandlers (2012),6,+_+Adam Pitzler,2012
9,+_+Roger Daltrey+_+John Entwistle+_+Keith Moon...,None,+_+USA,,+_+Music,None,,+_+English,None,7.7,2012-07-24,NaN,The Who: Quadrophenia - The Complete Story (2012),26,,2012
